In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.linalg import sqrtm
from copy import deepcopy

In [2]:
## load data
train = pd.read_csv('products.csv')
test = pd.read_csv('sampleSubmission.csv')

C:\Anaconda3\envs\aind-dog-copy\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
## fill missing values
#products['customerID'].fillna('BBID_0000', inplace=True)
train['promotion_description'].fillna('no_promo', inplace=True)
train['Gender'].fillna('no_gender', inplace=True)
train['State'].fillna('no_state', inplace=True)
train['PinCode'].fillna(-1, inplace=True)
train['DOB'].fillna("1", inplace=True)


In [4]:
#handling missing data for product_code
train = train[np.isfinite(train['product_code'])]

In [5]:
state_dict = {'MADHY PRADESH':'MADHYA PRADESH', 'TAMILNADU':'TAMIL NADU', 'MADHYA  PRADESH':'MADHYA PRADESH', 'HARAYANA':'HARYANA',
             'Jharkhand':'JHARKHAND','Tamilnadu':'TAMIL NADU','Tamil Nadu':'TAMIL NADU','Madhya Pradesh':'MADHYA PRADESH',
             'REST OF WEST BENGAL':'WEST BENGAL', 'west bengal':'WEST BENGAL','Uttar Pradesh':'UTTAR PRADESH', 'Delhi':'DELHI',
             'Bhopal':'BHOPAL','CHHATISGARH':'CHHATTISGARH','CHATTISGARH':'CHHATTISGARH', 'jharkhand':'JHARKHAND','Chandigarh':'CHANDIGARH',
             'UTTAR PRADESH WEST': 'UTTAR PRADESH','ODISHA':'ORISSA','MAHARASTRA':'MAHARASHTRA','madhya pradesh':'MADHYA PRADESH',
             'KARNATAK':'KARNATAKA','JAMMU and KASHMIR':'JAMMU AND KASHMIR','JAMMU KASHMIR':'JAMMU AND KASHMIR','Rajasthan':'RAJASTHAN',
             'east singhbhum':'JHARKHAND', 'ORRISA':'ORISSA','Andhra Pradesh':'ANDHRA PRADESH', 'UTTARANCHAL':'UTTARAKHAND',
             'Uttar pradesh':'UTTAR PRADESH','Maharashtra':'MAHARASHTRA','MP':'MADHYA PRADESH', 'UTTAR PRADESH EAST':'UTTAR PRADESH',
             'Punjab':'PUNJAB','maharashtra':'MAHARASHTRA','Karnataka':'KARNATAKA','M.P.':'MADHYA PRADESH','DAMAN':'DAMAN AND DIU',
             'HUBLI':'KARNATAKA','Tamil nadu':'TAMIL NADU','GUJRAT':'GUJARAT', 'Mp':'MADHYA PRADESH','Madhya pradesh':'MADHYA PRADESH',
             'West Bengal':'WEST BENGAL','Gujarat':'GUJARAT','UP':'UTTAR PRADESH','Chennai':'CHENNAI', 'm.p.':'MADHYA PRADESH',
             'kerala':'KERALA'}

train.replace({"State": state_dict}, inplace=True)

In [6]:
cnt_srs = train['product_code'].astype('int64').value_counts().reset_index().head(20)
#cnt_srs = products['product_code'].value_counts().reset_index()
cnt_srs.columns = ['product_code', 'frequency_count']
cnt_srs

,product_code,frequency_count
0,300776411,60994
1,300776410,58437
2,108037568,51338
3,300785148,41500
4,108100382,35259
5,300785150,34794
6,300840018,31149
7,108100362,27963
8,108005676,27727
9,300111517,25360


In [7]:
cnt_srs['product_code'].values

array([300776411, 300776410, 108037568, 300785148, 108100382, 300785150,
       300840018, 108100362, 108005676, 300111517, 108100306, 108000707,
       300776409, 300781593, 108100308, 300481673, 108100294, 108100290,
       108100352, 300070102], dtype=int64)

In [8]:
prod_index = []
for i in cnt_srs['product_code']:
    prod_index.append(i)

In [9]:
product_joined = pd.merge(train, cnt_srs, on='product_code', how='inner')
products_less = product_joined[['customerID', 'product_code']].reset_index()
products_less.drop('index', axis=1, inplace=True)
products_to_join = pd.DataFrame(train['customerID']).reset_index()
products_to_join.drop('index', axis=1, inplace=True)

In [10]:
# Join the dummy variables to the main dataframe
train = pd.merge(products_to_join, products_less.drop_duplicates(subset=['customerID'], keep='last'), on='customerID', how='left')
train.head()

,customerID,product_code
0,BBID_2041,1.081e+08
1,BBID_2041,1.081e+08
2,BBID_2041,1.081e+08
3,BBID_2041,1.081e+08
4,BBID_2041,1.081e+08


In [11]:
train.dropna(inplace=True)

In [12]:
prod_dict = {'_'}

train = pd.get_dummies(train, columns=['product_code'], prefix=prod_dict)
train.columns=train.columns.str.replace('_','')
train.reset_index(drop=True).head()

,customerID,108000707.0,108005676.0,108037568.0,108100290.0,108100294.0,108100306.0,108100308.0,108100352.0,108100362.0,...,300070102.0,300111517.0,300481673.0,300776409.0,300776410.0,300776411.0,300781593.0,300785148.0,300785150.0,300840018.0
0,BBID_2041,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BBID_2041,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,BBID_2041,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,BBID_2041,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,BBID_2041,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
test = test[['customerID']]

In [14]:
true = deepcopy(train)

In [15]:
users = true['customerID'].tolist()
true.drop('customerID', axis=1, inplace=True)

items = true.columns.tolist()
u = {}
for i in range(len(users)):
    u[users[i]] = i

trueMat = np.array(true)
print("users dict formed")

users dict formed


In [16]:
def reorder(train):
    train.index = train['customerID'].tolist()
    train.drop('customerID', axis=1, inplace=True)
    train = train.reindex(users)
    return train

In [17]:
train = reorder(train)

In [18]:
train

,108000707.0,108005676.0,108037568.0,108100290.0,108100294.0,108100306.0,108100308.0,108100352.0,108100362.0,108100382.0,300070102.0,300111517.0,300481673.0,300776409.0,300776410.0,300776411.0,300781593.0,300785148.0,300785150.0,300840018.0
BBID_2041,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BBID_2041,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BBID_2041,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BBID_2041,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BBID_2041,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BBID_2041,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BBID_2041,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BBID_2041,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BBID_2041,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BBID_2041,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
def svd(train, trueMat, k):
    utilMat = np.array(train)

    mask = np.isnan(utilMat)
    masked_arr=np.ma.masked_array(utilMat, mask)
    item_means=np.mean(masked_arr, axis=0)
    utilMat = masked_arr.filled(item_means)

    x = np.tile(item_means, (utilMat.shape[0],1))

    utilMat = utilMat - x

    print(utilMat)

    U, s, V=np.linalg.svd(utilMat, full_matrices=False)
    s=np.diag(s)
    s=s[0:k,0:k]
    U=U[:,0:k]
    V=V[0:k,:]

    s_root=sqrtm(s)

    Usk=np.dot(U,s_root)
    skV=np.dot(s_root,V)
    UsV = np.dot(Usk, skV)

    UsV = UsV + x

    UsV = np.ma.masked_where(trueMat==1,UsV).filled(fill_value=-999)

    #print(UsV)
    print("svd done")
    return UsV

In [20]:
def max_items(UsV,x,j):
    out = []

    for xx in x:
        if UsV[j,xx]>0.001: # setting a threshold
            out.append(items[xx])
        
    if len(out) < 20:
        for j in range(20 - len(out)):
            out.append("None")
        

    return out




In [21]:
def recommend(test):

    UsV = (5*svd(train=train,trueMat=trueMat,k=13) + 5*svd(train=train, trueMat=trueMat, k=13))/10

    pred = []
    testusers = test['customerID'].tolist()

    n = 20
    for user in testusers[:]:
        if user not in u:
            pred.append("None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None")
            test['products'] = pd.Series(pred)
        else:
            j = u[user]
            p = max_items(UsV, UsV[j,:].argsort()[-n:][::-1], j)
            pred.append(",".join(p))
            print(len(p))
            test['products'] = pd.Series(pred)
    #test['products'] = pred
    test.to_csv('sub_new_k13_upd.csv', index=False)


In [22]:
recommend(test)

[[-0.25373398231527766 -0.18531923555947408 -0.07815014466205575 ...,
  -0.03511880050332124 -0.01646199625077039 -0.04314596636821201]
 [-0.25373398231527766 -0.18531923555947408 -0.07815014466205575 ...,
  -0.03511880050332124 -0.01646199625077039 -0.04314596636821201]
 [-0.25373398231527766 -0.18531923555947408 -0.07815014466205575 ...,
  -0.03511880050332124 -0.01646199625077039 -0.04314596636821201]
 ..., 
 [0.7462660176847223 -0.18531923555947408 -0.07815014466205575 ...,
  -0.03511880050332124 -0.01646199625077039 -0.04314596636821201]
 [0.7462660176847223 -0.18531923555947408 -0.07815014466205575 ...,
  -0.03511880050332124 -0.01646199625077039 -0.04314596636821201]
 [0.7462660176847223 -0.18531923555947408 -0.07815014466205575 ...,
  -0.03511880050332124 -0.01646199625077039 -0.04314596636821201]]
svd done
[[-0.25373398231527766 -0.18531923555947408 -0.07815014466205575 ...,
  -0.03511880050332124 -0.01646199625077039 -0.04314596636821201]
 [-0.25373398231527766 -0.18531923555

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix

svd = TruncatedSVD(n_components=14, n_iter=7, random_state=42)
svd.fit(train)  


In [ ]:
print(svd.explained_variance_ratio_)  

print(svd.explained_variance_ratio_.sum())  

#print(svd.singular_values_)  